<a href="https://colab.research.google.com/github/GerardJLarkin/MachineLearning/blob/master/handmade_decision_tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Manual development of a Decision Tree learning algorithm

In [1]:
# upgrade gspread: only need to run if you run into authorization issues
#!pip install --upgrade gspread

In [2]:
# authorise the importing of data from my own google drive
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

worksheet = gc.open('beer.txt').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
#print(rows)

# Convert to a DataFrame and render.
import pandas as pd
d1 = pd.DataFrame(rows)

import warnings
warnings.filterwarnings('ignore')

from operator import itemgetter
import numpy as np

In [3]:
# add column/feature headings to dataset
# this is a manual step required with our dataset
d1.columns = ['calorific_value', 'nitrogen', 'turbidity', 'style', 'alcohol', \
              'sugars', 'bitterness', 'beer_id', 'colour', \
              'degree_of_fermentation']
#print(type(d1))

In [4]:
# put target column to the end of the dataframe for ease of access
# this is a manual step required for our dataset
d1 = d1[['calorific_value','nitrogen', 'turbidity', 'alcohol','sugars', \
         'bitterness', 'beer_id', 'colour', 'degree_of_fermentation','style']]

## Preprocessing steps

In [5]:
# After reviewing the dataset I identified the column beer_id as a feature of unique values
# These will not be great for decision making as there is no relationship within the feature
# that corresponds to the different beer styles, it will therefore be removed
d1 = d1[['calorific_value','nitrogen', 'turbidity', 'alcohol','sugars', \
         'bitterness','colour', 'degree_of_fermentation','style']]

In [6]:
# split data into train and test sets
train=d1.sample(frac=0.66667, random_state=1)
test=d1.drop(train.index)

In [7]:
# split training and test data into features and target
X_train = train.iloc[:,:-1].astype(float)
y_train = pd.DataFrame(train.iloc[:,-1]).astype('category')

X_test = test.iloc[:,:-1].astype(float)
y_test = pd.DataFrame(test.iloc[:,-1]).astype('category')


In [8]:
# select k best features by getting the variance in each feature
# it is important to note that scaling a feature also changes its variance so 
# it's necessary to determine variance on the non-scaled dataset to see which
# (if any) feature is appropriate to remove
col_nam = list(X_train.columns)
col_vars = list(X_train.var())
col_nam_var = list(zip(col_nam, col_vars))
# set variance threshold at arbirary value of 0.5
column_name = [item[0] for item in col_nam_var if item[1] > 0.5] 
X_train = X_train[column_name]

# select the same features for the test dataset
X_test = X_test[column_name]

In [9]:
# scale each feature in the training & tests sets to remove bias on dominant 
# features
X_train_norm = (X_train - X_train.min()) / (X_train.max() - X_train.min())
X_test_norm = (X_test - X_test.min()) / (X_test.max() - X_test.min())

## Build the Decision Tree Algorithm

In [10]:
# create function to split dataset on maximum gain ratio
# logic for this function derived from explanation of algorithm at following link
# https://sefiks.com/2018/05/13/a-step-by-step-c4-5-decision-tree-example/
def gain_ratio_split(features, target):
    # calculate entropy of target labels before split
    target_ent = -sum((target.value_counts()/len(target)) * np.log2((target.value_counts()/len(target))))

    gainratios = []
    num_cols = features.shape[1]
    # get index of columns
    for i in range(num_cols):
        # get column name via its index
        column_name = features.columns[i]
        # call input dataframe again and get column to perform entropy calc on
        single_column_df = pd.DataFrame(features[column_name])
        # add target lables to dataframe to calucluate entropy
        duo_column_df = single_column_df.merge(target, left_index=True, right_index=True).sort_values(by=column_name)
        # get number of rows to create iterative sub frames to perform comnbined entropy over
        for j in range(len(duo_column_df)):
            # this call gets the number of rows starting at the beginning corresponding to interative index
            sub_df_1 = duo_column_df.head(j)
            # this call gets the remainder of the dataframe excluded from the previous call
            sub_df_2 = duo_column_df.tail(len(duo_column_df)-j)
            # sense check if either sub df is empty then pass as errors will occur
            if sub_df_1.empty or sub_df_2.empty:
                pass
            else:
                # find the mid point value between the two subsets (max of lower subset, min of upper subset)
                mp_val = ((sub_df_1.max(axis = 0)[0] + sub_df_2.min(axis = 0)[0]) / 2)  
                # calculate entropy for both sub frames
                sub_df_1_ent = -sum(sub_df_1.iloc[:,1].value_counts()/len(sub_df_1) * np.log2(sub_df_1.iloc[:,1].value_counts()/len(sub_df_1)))
                sub_df_2_ent = -sum(sub_df_2.iloc[:,1].value_counts()/len(sub_df_2) * np.log2(sub_df_2.iloc[:,1].value_counts()/len(sub_df_2)))
                # calculate information gain by from each of the sub frame splits
                gain = target_ent - ((len(sub_df_1)/len(duo_column_df)) * sub_df_1_ent) - ((len(sub_df_2)/len(duo_column_df)) * sub_df_2_ent)
                # calculate split info to normalise gain (remove bias)
                splitinfo = -( (len(sub_df_1)/len(duo_column_df)) * np.log2( (len(sub_df_1)/len(duo_column_df)) )) - \
                             ( (len(sub_df_2)/len(duo_column_df)) * np.log2( (len(sub_df_2)/len(duo_column_df)) ))
                # gain ratio calculation 
                gainratio = np.nan_to_num(gain / splitinfo)
                gainratios.append((column_name, mp_val, gainratio))
    # get feature, feature value and IG to perform split on by getting max gain ratio across all features
    try:
        out = max(gainratios,key=itemgetter(2))
    except:
        out = (None, None, None)

    return out

In [11]:
# create function to calculate node impurity
def node_impurity(node_data):
    imp_val = list(node_data.iloc[:,-1].value_counts()/ len(node_data))
    # if the length of imp_val is greater than 1 it contains multiple classes
    if len(imp_val) == 1:
        return True
    else:
        return False

In [12]:
def leaf_classifier(node_data):
    return node_data.iloc[:,-1].value_counts().idxmax()

In [13]:
# After multiple iterations I found that a min tree depth of 6 returns the highest
# accuracy, above 6 the accuracy does not increase. my model does not run with a 
# depth of 5, 2, or 1. depth of 4 and 3 have a lower accuracy than 6
def dec_t(features, target, depth = 0, stop = 6):
    stop = stop
    target = target
    depth = depth  
    data = features.merge(target, left_index = True, right_index = True)
    n_imp = node_impurity(data)

    # check to see if we are at final recursion or if node is pure, if so classify node
    if (n_imp == True) or (depth == stop) or (features.empty):
        return leaf_classifier(data)
    # If the data is not pure or we are still within the max number of recursions:
    else:
        # call split function to get feature and value to split on
        column_name, mp_val, gainratio = gain_ratio_split(features, target)
        if column_name is None:
            return leaf_classifier(data)
        else:
            # create string of feature & split values to design tree
            split = ("{} {}").format(column_name, mp_val)
            # create dictionary to append edges and their values
            edge = {split : []}
            depth += 1
            # get child datasets after split
            # there is a question as to whether or not to remove the feature that the data is being split
            # on so it will not be available for the next iteration. If I do not drop them my algorthim
            # continuously uses the same feature but splits at different values, if I exclude my algorithm
            # choses a new feature each time. 
            # I am choosing to drop a feature at each recursion: https://machinelearningmastery.com/rfe-feature-selection-in-python/
            child1 = features[features[column_name] >= mp_val].drop(columns = column_name)
            child2 = features[features[column_name] < mp_val].drop(columns = column_name)
            # get apprpritate target values for new child datasets
            tar1 = pd.DataFrame(child1.merge(target, left_index=True, right_index=True).iloc[:,-1])
            tar2 = pd.DataFrame(child2.merge(target, left_index=True, right_index=True).iloc[:,-1])
            # begin recursion and append recursive output to edges dictionary to print tree
            left = dec_t(child1, tar1, depth = depth, stop = stop)
            right = dec_t(child2, tar2, depth = depth, stop = stop)
            if left != right:
                edge[split].append(left)
                edge[split].append(right)
            
            return edge


In [14]:
from pprint import pprint
t = dec_t(X_train_norm, y_train)
pprint(t)

{'bitterness 0.3445410409184757': [{'calorific_value 0.015075376570541452': [{'turbidity 0.017860609564024174': [{'sugars 0.07331975560081494': [{'colour 0.2938144329896908': [{'degree_of_fermentation 0.2946390462061134': ['ale',
                                                                                                                                                                                                                               'lager']},
                                                                                                                                                                                'lager']},
                                                                                                                                                 'lager']},
                                                                                                                 'lager']},
                                                                     

In [15]:
def unseen(data, tree):
    # extract the first key of the tree dictionary I built in the previous
    # function
    edge = list(tree.keys())[0]
    # extract feature and split value
    feature, mp_val = edge.split(sep = ' ')
    # classify unseen data based on same criteria as decision tree function
    if data[feature] >= float(mp_val):
        # check if extraced key is still in dict format, if so apply function again
        # if not we've reached our label
        if isinstance(tree[edge][0], dict) == True:
            return unseen(data, tree[edge][0])
        else:
            return tree[edge][0]
    else:
        if isinstance(tree[edge][1], dict) == True:
            return unseen(data, tree[edge][1])
        else:
            return tree[edge][1]

def predictor(data, tree):
    # In the docs its states the args need to be a tuple, if I leave one argument (tree) 
    # its treated like a str so after many attempts I have to insert the comma to give
    # a null second element of a tuple
    data['style'] = data.apply(unseen, axis = 1, args = (tree, ))

    return data


In [16]:
# create a copy of the test feature set
test_cases = X_test_norm.copy()
# merge it with the test target set to get real labels
test_with_label = test_cases.merge(y_test, left_index = True, right_index = True)

In [17]:
# predict target labels from predictor function
pred = predictor(test_cases, t)

In [18]:
# find accuracy of model prediction
correct_labels = test_with_label.merge(pred, how = 'inner', suffixes = ('t_', 'p_'))
accuracy = '{:.2f}'.format(len(correct_labels)/len(test_with_label) * 100) + '%'
accuracy

'70.59%'

In [19]:
# cross validate model by selecting 10 random subsets of test data
def rand_pred_test(test, tree):
    accuracy_values = []
    for i in range(10):
        sub_df = test.sample(frac = 0.2)
        test_with_label = sub_df.merge(y_test, left_index = True, right_index = True)
        pred = predictor(sub_df, tree)
        correct_labels = test_with_label.merge(pred, how = 'inner', suffixes = ('t_', 'p_'))
        accuracy = len(correct_labels)/len(test_with_label)
        accuracy_values.append(accuracy)

    return np.mean(accuracy_values)

In [20]:
# get average accuracy of model after predicting 10 subsets of the test dataset
# as the subsets change each time, the average accuracy changes after each run
avg_accuracy = '{:.2f}'.format(rand_pred_test(X_test_norm, t) * 100) + '%'
avg_accuracy

'72.00%'

In [21]:
# choose reference algorithm to assess accuracy of handmade model
# as I coded up an implementation of te C4.5 algorithm, the closest algorithim
# I could find in the sci-kit leanrn libraries was the CART decision tree classifier
# I will implement this for comparision
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier()
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)

from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[14  1  0]
 [ 2 17  1]
 [ 0  0 16]]
              precision    recall  f1-score   support

         ale       0.88      0.93      0.90        15
       lager       0.94      0.85      0.89        20
       stout       0.94      1.00      0.97        16

    accuracy                           0.92        51
   macro avg       0.92      0.93      0.92        51
weighted avg       0.92      0.92      0.92        51

